In [ ]:
!pip install ultralytics -qq
!pip install pylabel -qq
!pip install lxml tqdm -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.0 MB/s eta 0:00:00


In [ ]:
import zipfile
import os, random, cv2, json, shutil, yaml
import matplotlib.pyplot as plt
import ultralytics

from google.colab import drive
from ultralytics import YOLO
from pylabel import importer
from tqdm import tqdm
from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# import json
# from collections import defaultdict
# from PIL import Image

# # ========== EDIT THESE PATHS ==========
# # JSONs
# train_json = "/content/drive/MyDrive/COCO2017/annotations/filtered_train2017.json"
# val_json   = "/content/drive/MyDrive/COCO2017/annotations/filtered_val2017.json"

# # lists (one filename per line, basename or path)
# train_list_path = "/content/drive/MyDrive/COCO_FILTERD/txt/train_images.txt"
# val_list_path   = "/content/drive/MyDrive/COCO_FILTERD/txt/val_images.txt"

# # image folders
# train_images_dir = "/content/drive/MyDrive/COCO_FILTERD/images/train"
# val_images_dir   = "/content/drive/MyDrive/COCO_FILTERD/images/val"

# # output label folders (will contain /labels subfolder)
# out_root_train = "/content/drive/MyDrive/COCO_FILTERD/labels/train"
# out_root_val   = "/content/drive/MyDrive/COCO_FILTERD/labels/val"

# # shared mapping output (we save it under a common parent)
# mapping_out_dir = "/content/drive/MyDrive/COCO_FILTERD/labels"  # will contain coco_to_yolo.json, yolo_to_coco.json, names.txt
# mapping_path = os.path.join(mapping_out_dir, "coco_to_yolo.json")
# # ======================================

# os.makedirs(out_root_train, exist_ok=True)
# os.makedirs(out_root_val, exist_ok=True)
# os.makedirs(mapping_out_dir, exist_ok=True)

# def load_list(path):
#     if not path or not os.path.exists(path):
#         return []
#     items = []
#     with open(path, "r") as f:
#         for line in f:
#             s = line.strip()
#             if s:
#                 items.append(os.path.basename(s))
#     return items

# def load_coco(path):
#     if not path or not os.path.exists(path):
#         return {"images": [], "annotations": [], "categories": []}
#     with open(path, "r") as f:
#         return json.load(f)

# def build_or_load_mapping(coco_list, mapping_path, mapping_out_dir):
#     # try load existing mapping
#     if os.path.exists(mapping_path):
#         try:
#             with open(mapping_path, "r") as f:
#                 coco_to_yolo = json.load(f)
#             print("Loaded existing mapping:", mapping_path)
#             return coco_to_yolo, None, None
#         except Exception as e:
#             print("Failed to load mapping, will rebuild. Error:", e)

#     # build mapping from union of categories across provided cocos
#     cats_union = {}
#     for coco in coco_list:
#         for c in coco.get("categories", []):
#             cats_union[c["id"]] = c["name"]

#     # sort by category id for stability
#     cats_sorted = sorted(cats_union.items(), key=lambda x: int(x[0]))

#     coco_to_yolo = {}
#     yolo_to_coco = {}
#     names = []
#     for new_idx, (cid, name) in enumerate(cats_sorted):
#         coco_to_yolo[str(cid)] = new_idx
#         yolo_to_coco[str(new_idx)] = int(cid)
#         names.append(name)

#     # save mapping + names
#     with open(os.path.join(mapping_out_dir, "coco_to_yolo.json"), "w") as f:
#         json.dump(coco_to_yolo, f, indent=2)
#     with open(os.path.join(mapping_out_dir, "yolo_to_coco.json"), "w") as f:
#         json.dump(yolo_to_coco, f, indent=2)
#     with open(os.path.join(mapping_out_dir, "names.txt"), "w") as f:
#         f.write("\n".join(names))

#     print(f"Built mapping with {len(names)} categories and saved under {mapping_out_dir}")
#     return coco_to_yolo, yolo_to_coco, names

# def generate_labels_for_list(coco, fname_to_id, anns_by_img, names_map, coco_to_yolo, images_dir, list_names, labels_out_dir, missing_json_list_path, missing_image_list_path):
#     os.makedirs(labels_out_dir, exist_ok=True)
#     written = 0
#     missing_in_json = []
#     missing_image_file = []

#     for fname in list_names:
#         base = os.path.splitext(fname)[0]
#         out_path = os.path.join(labels_out_dir, base + ".txt")

#         if fname not in fname_to_id:
#             missing_in_json.append(fname)
#             open(out_path, "w").close()
#             continue

#         img_id = fname_to_id[fname]
#         iminfo = coco["images_by_id"].get(img_id)
#         if iminfo is None:
#             missing_in_json.append(fname)
#             open(out_path, "w").close()
#             continue

#         W = iminfo.get("width")
#         H = iminfo.get("height")

#         # try to get size from disk if missing
#         if (W is None or H is None):
#             img_path = os.path.join(images_dir, fname)
#             if os.path.exists(img_path):
#                 try:
#                     with Image.open(img_path) as im:
#                         W, H = im.size
#                 except Exception:
#                     missing_image_file.append(fname)
#                     open(out_path, "w").close()
#                     continue
#             else:
#                 missing_image_file.append(fname)
#                 open(out_path, "w").close()
#                 continue

#         lines = []
#         for ann in anns_by_img.get(img_id, []):
#             x, y, w, h = ann["bbox"]
#             if w <= 0 or h <= 0:
#                 continue
#             coco_cat = ann["category_id"]
#             if str(coco_cat) in coco_to_yolo:
#                 cls = int(coco_to_yolo[str(coco_cat)])
#             else:
#                 cls = int(coco_cat)
#             cx = (x + w/2) / float(W)
#             cy = (y + h/2) / float(H)
#             wn = w / float(W)
#             hn = h / float(H)
#             # clamp
#             cx = max(0.0, min(1.0, cx))
#             cy = max(0.0, min(1.0, cy))
#             wn = max(0.0, min(1.0, wn))
#             hn = max(0.0, min(1.0, hn))
#             lines.append(f"{cls} {cx:.6f} {cy:.6f} {wn:.6f} {hn:.6f}")

#         with open(out_path, "w") as f:
#             f.write("\n".join(lines))
#         written += 1

#     # save missing lists
#     if missing_in_json and missing_json_list_path:
#         with open(missing_json_list_path, "w") as f:
#             for n in missing_in_json:
#                 f.write(n + "\n")
#     if missing_image_file and missing_image_list_path:
#         with open(missing_image_list_path, "w") as f:
#             for n in missing_image_file:
#                 f.write(n + "\n")

#     return written, missing_in_json, missing_image_file

# # -------- Load lists ----------
# train_names = load_list(train_list_path)
# val_names = load_list(val_list_path)

# print(f"Train list: {len(train_names)} entries (unique {len(set(train_names))})")
# print(f"Val list:   {len(val_names)} entries (unique {len(set(val_names))})")

# # -------- Load COCO JSONs ----------
# train_coco = load_coco(train_json)
# val_coco   = load_coco(val_json)

# # Build quick maps for train and val
# def prepare_coco_maps(coco):
#     images = coco.get("images", [])
#     anns = coco.get("annotations", [])
#     images_by_id = {im["id"]: im for im in images}
#     fname_to_id = {os.path.basename(im["file_name"]): im["id"] for im in images}
#     anns_by_img = defaultdict(list)
#     for ann in anns:
#         anns_by_img[ann["image_id"]].append(ann)
#     coco["images_by_id"] = images_by_id
#     coco["fname_to_id"] = fname_to_id
#     coco["anns_by_img"] = anns_by_img

# prepare_coco_maps(train_coco)
# prepare_coco_maps(val_coco)

# # -------- Build or load mapping from union of categories ----------
# coco_to_yolo, yolo_to_coco, names = build_or_load_mapping([train_coco, val_coco], mapping_path, mapping_out_dir)

# # if build_or_load returned only coco_to_yolo (when loaded), attempt to also build names from json(s)
# if names is None:
#     # try to collect category names from train then val
#     cats = train_coco.get("categories", []) or val_coco.get("categories", [])
#     cats_sorted = sorted(cats, key=lambda c: c["id"]) if cats else []
#     names = [c["name"] for c in cats_sorted] if cats_sorted else []

# # -------- Generate labels for train ----------
# written_train, missing_train_json, missing_train_image = generate_labels_for_list(
#     train_coco,
#     train_coco["fname_to_id"],
#     train_coco["anns_by_img"],
#     names,
#     coco_to_yolo,
#     train_images_dir,
#     train_names,
#     os.path.join(out_root_train, "labels"),
#     os.path.join(out_root_train, "missing_in_json.txt"),
#     os.path.join(out_root_train, "missing_image_files.txt")
# )

# print(f"\nTrain: Written {written_train} labels to {os.path.join(out_root_train, 'labels')}")
# if missing_train_json:
#     print(f"Train: {len(missing_train_json)} images in train_list not found in train.json (sample): {missing_train_json[:10]}")
# if missing_train_image:
#     print(f"Train: {len(missing_train_image)} train images missing on disk or unreadable (sample): {missing_train_image[:10]}")

# # -------- Generate labels for val ----------
# written_val, missing_val_json, missing_val_image = generate_labels_for_list(
#     val_coco,
#     val_coco["fname_to_id"],
#     val_coco["anns_by_img"],
#     names,
#     coco_to_yolo,
#     val_images_dir,
#     val_names,
#     os.path.join(out_root_val, "labels"),
#     os.path.join(out_root_val, "missing_in_json.txt"),
#     os.path.join(out_root_val, "missing_image_files.txt")
# )

# print(f"\nVal: Written {written_val} labels to {os.path.join(out_root_val, 'labels')}")
# if missing_val_json:
#     print(f"Val: {len(missing_val_json)} images in val_list not found in val.json (sample): {missing_val_json[:10]}")
# if missing_val_image:
#     print(f"Val: {len(missing_val_image)} val images missing on disk or unreadable (sample): {missing_val_image[:10]}")

# print("\nAll done.")


In [ ]:
# output_path = "/content/drive/MyDrive/COCO_FILTERD/data.yaml"
# root_path   = "/content/drive/MyDrive/COCO_FILTERD"

# classes = ["person","chair","car","dining table","cup","bottle","bowl","handbag","truck","bench","backpack","book",
#            "cell phone","tv","couch","dog","knife","sports ball","traffic light","cat","umbrella","bus","bed","train",
#            "fork","spoon","laptop","motorcycle","bicycle","sandwich","banana","stop sign","orange","carrot","apple"
#            ]

# data_yaml = {
#     "path": root_path,
#     "train": "images/train",
#     "val": "images/val",
#     "nc": len(classes),
#     "names": classes
# }

# with open(output_path, "w") as f:
#     yaml.dump(data_yaml, f, sort_keys=False)

# print("data.yaml created at:", output_path)
# print("Number of classes =", len(classes))


In [ ]:
root = "/content/drive/MyDrive/COCO_FILTERD"

train_images = os.path.join(root, "images/train")
train_labels = os.path.join(root, "labels/train")

val_images = os.path.join(root, "images/val")
val_labels = os.path.join(root, "labels/val")

def count_files(folder, exts):
    if not os.path.exists(folder):
        return 0
    return len([f for f in os.listdir(folder) if f.lower().endswith(exts)])

img_exts = (".jpg", ".jpeg", ".png")

print("===== TRAIN =====")
print("train lenth:", count_files(train_images, img_exts))
print("labels train lenth:", count_files(train_labels, (".txt",)))

print("\n===== VAL =====")
print("val lenth:", count_files(val_images, img_exts))
print("labels val lenth:", count_files(val_labels, (".txt",)))

===== TRAIN =====
train lenth: 59637


In [ ]:
import argparse
import sys
from pathlib import Path
from ultralytics import YOLO


# ---------- CUSTOM PATHS ----------
DEFAULT_DATA_YAML = "/content/drive/MyDrive/COCO_FILTERD/data.yaml"
DEFAULT_SAVE_DIR = "/content/drive/MyDrive/COCO_FILTERD/saved_models"
# -----------------------------------


def train_youreyes_model(
    model_size: str = "n",
    data_yaml: str = DEFAULT_DATA_YAML,
    epochs: int = 50,
    batch_size: int = 16,
    img_size: int = 640,
    project_name: str = "youreyes_model",
    resume: bool = False,
    save_dir: str = DEFAULT_SAVE_DIR,
):
    """
    Train YOLO model on Your Eyes dataset
    """
    print("=" * 70)
    print("Your Eyes - YOLO Model Training")
    print("=" * 70)
    print()

    # Validate data.yaml exists
    if not Path(data_yaml).exists():
        print(f"❌ Error: {data_yaml} not found!")
        print("Please create data.yaml with your dataset configuration")
        return

    # Select base model
    model_name = f"yolov8{model_size}.pt"
    print(f"📦 Base model: {model_name}")
    print(f"📊 Dataset config: {data_yaml}")
    print(f"🔄 Epochs: {epochs}")
    print(f"📏 Batch size: {batch_size}")
    print(f"🖼️  Image size: {img_size}")
    print(f"📁 Project name: {project_name}")
    print(f"📂 Save dir: {save_dir}")
    print()

    # Load model
    print("Loading base model...")
    model = YOLO(model_name)

    # Create save dir path if not exists
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    # Training parameters
    train_args = {
        "data": data_yaml,
        "epochs": epochs,
        "imgsz": img_size,
        "batch": batch_size,
        "name": project_name,
        "project": save_dir,
        "exist_ok": True,
        "patience": 10,
        "save": True,
        "save_period": 10,
        "cache": False,
        "device": 0,
        "workers": 8,
        "pretrained": True,
        "optimizer": "auto",
        "verbose": True,
        "seed": 42,
        "deterministic": True,
        "single_cls": False,
        "rect": False,
        "cos_lr": False,
        "close_mosaic": 10,
        "resume": resume,
        "amp": True,
        "fraction": 1.0,
        "profile": False,
        "overlap_mask": True,
        "mask_ratio": 4,
        "dropout": 0.0,
        "val": True,
        "plots": True,
    }

    print("🚀 Starting training...")
    print("=" * 70)
    print()

    try:
        results = model.train(**train_args)

        print()
        print("=" * 70)
        print("✅ Training completed successfully!")
        print("=" * 70)
        print()

        run_folder = Path(save_dir) / project_name
        print(f"📁 Results saved to: {run_folder}")
        print(f"🏆 Best model: {run_folder}/weights/best.pt")
        print(f"📊 Last model: {run_folder}/weights/last.pt")
        print()
        print("To use your trained model:")
        print("1. In the app, go to Settings")
        print(f"2. Set model path to: {run_folder}/weights/best.pt")
        print("3. Click 'Load/Reload Model'")
        print()

    except Exception as e:
        print()
        print("=" * 70)
        print(f"❌ Training failed: {e}")
        print("=" * 70)
        print()


def validate_model(model_path: str, data_yaml: str = DEFAULT_DATA_YAML):
    """
    Validate trained model on validation set
    """
    print("=" * 70)
    print("Validating Model")
    print("=" * 70)
    print()

    model = YOLO(model_path)
    results = model.val(data=data_yaml)

    print()
    print("Validation Results:")
    # depending on ultralytics version the attribute names can vary; guard for missing attrs
    try:
        print(f"mAP50: {results.box.map50:.4f}")
    except Exception:
        print("mAP50: (attribute not found in results)")

    try:
        print(f"mAP50-95: {results.box.map:.4f}")
    except Exception:
        print("mAP50-95: (attribute not found in results)")
    print()


def build_arg_parser():
    parser = argparse.ArgumentParser(description="Train Your Eyes YOLO model")

    parser.add_argument("--model", type=str, default="n", choices=["n", "s", "m", "l", "x"])
    parser.add_argument("--data", type=str, default=DEFAULT_DATA_YAML, help="Path to data.yaml")
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch", type=int, default=16)
    parser.add_argument("--img-size", type=int, default=640)
    parser.add_argument("--name", type=str, default="youreyes_model")
    parser.add_argument("--resume", action="store_true")
    parser.add_argument("--validate", type=str, help="Validate a trained model (provide path to weights)")
    parser.add_argument("--save-dir", type=str, default=DEFAULT_SAVE_DIR, help="Directory to save training runs")

    return parser


if __name__ == "__main__":
    parser = build_arg_parser()

    # -------------------------
    # NOTE: use parse_known_args() to avoid Jupyter/Colab kernel args errors.
    # This will work both in notebooks and when you run as a script.
    # -------------------------
    args, unknown = parser.parse_known_args()
    # You can inspect unknown if you want: print("Ignored args:", unknown)

    if args.validate:
        validate_model(args.validate, args.data)
    else:
        train_youreyes_model(
            model_size=args.model,
            data_yaml=args.data,
            epochs=args.epochs,
            batch_size=args.batch,
            img_size=args.img_size,
            project_name=args.name,
            resume=args.resume,
            save_dir=args.save_dir,
        )


Your Eyes - YOLO Model Training

📦 Base model: yolov8n.pt
📊 Dataset config: /content/drive/MyDrive/COCO_FILTERD/data.yaml
🔄 Epochs: 50
📏 Batch size: 16
🖼️  Image size: 640
📁 Project name: youreyes_model
📂 Save dir: /content/drive/MyDrive/COCO_FILTERD/saved_models

Loading base model...
🚀 Starting training...

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/COCO_FILTERD/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou

KeyboardInterrupt: 